In [1]:
import tensorflow as tf
import pandas as pd 
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import random
from sklearn.decomposition import KernelPCA
import gc

In [2]:
y_raw=pd.read_csv(r"C:\Users\ksi03\Downloads\RP_data\train_label.csv", sep=',')
def variable_extracter(hist):
    loss = hist.history['loss'][0]
    acc=hist.history['accuracy'][0]

    return loss, acc

def scalepca_pipeline(x, scaler):
    x=scaler.transform(x)
    #x=pca.transform(x)
    return x

def batch_generater(filenum, scaler, nrows=None, rate=0.9, root=r"C:\Users\ksi03\Downloads\RP_data\train\\",):
    y_f=[]
    if nrows==None:
        x_raw=pd.read_csv(root+str(filenum)+".csv", sep=',').values
        #x_f=scalepca_pipeline(x_raw, scaler, pca)
        x_f=scaler.transform(x_raw)
        
        y=y_raw.iloc[filenum, 1]
        loc=10
        for _ in range(loc):
            y_f.append(198)

        for i in range(x_f.shape[0]-loc):
            y_f.append(y)
    else:
        x_raw=pd.read_csv(root+str(filenum)+".csv", sep=',', nrows=nrows).values
        #x_f=scalepca_pipeline(x_raw, scaler, pca)
        x_f=scaler.transform(x_raw)
        
        y=y_raw.iloc[filenum, 1]
        loc=10
        for stat_a in range(loc):
            y_f.append(198)

        for stat_b in range(nrows-loc):
            y_f.append(y)

    
    y_f=np.asarray(y_f)
    gc.collect()
    return x_f, y_f


def scaler_pca_generater( nrows, ref_filenum=100, dim=1000):
    #trainset인 0~745 이내에서 샘플을 100개 추출해 scaler와 pca 제작
    filelist=random.sample(range(0, 745), ref_filenum)
    scaler=MinMaxScaler()
    kpca = KernelPCA(n_components=dim, kernel='rbf')
    x_sample=[]
    i=False
    for num in filelist:
        if num!=30:
            if i!=False:
                tmp=pd.read_csv(r"C:\Users\ksi03\Downloads\RP_data\train\\"+str(num)+".csv", sep=',', nrows=nrows)
                tmp=tmp.iloc[10:]
                x_sample=data_raw=pd.concat([x_sample, tmp], ignore_index=True)
            else:
                x_sample=pd.read_csv(r"C:\Users\ksi03\Downloads\RP_data\train\\"+str(num)+".csv", sep=',', nrows=nrows)
        
    scaler.fit(x_sample)
    #kpca.fit(x_sample)
    kpca=False
    
    del x_sample
    gc.collect()
    return filelist, scaler


In [3]:
_, scaler=scaler_pca_generater(nrows=30, ref_filenum=300)
gc.collect()

0

In [4]:
#828개의 dataset, 745개는 train, 43개는 valid, 40개는 test
#valid, train=상주변수 
#train=가변변수
y_raw=pd.read_csv(r"C:\Users\ksi03\Downloads\RP_data\train_label.csv", sep=',')
for i in range(746, 788):
    if i==746: 
        x_t, y_t=batch_generater(i, scaler,  nrows=30)
        x_t=x_t[10:]
        y_t=y_t[10:]
    else: 
        x_t2, y_t2=batch_generater(i, scaler, nrows=30)
        x_t2=x_t2[10:]
        y_t2=y_t2[10:]
        
        x_t=np.concatenate((x_t, x_t2))
        y_t=np.concatenate((y_t, y_t2))
        
y_valid=y_t
x_valid=x_t



In [4]:
for i in range(789, 828):
    if i==789: 
        x_t, y_t=batch_generater(i, scaler, nrows=30)
        x_t=x_t[10:]
        y_t=y_t[10:]
    else: 
        x_t2, y_t2=batch_generater(i, scaler, nrows=30)
        x_t2=x_t2[10:]
        y_t2=y_t2[10:]
        x_t=np.concatenate((x_t, x_t2))
        y_t=np.concatenate((y_t, y_t2))
        
y_test=y_t
x_test=x_t

In [10]:
x_valid=x_valid.reshape(x_valid.shape[0], 1, x_valid.shape[1])


NameError: name 'x_valid' is not defined

In [6]:


print(x_valid.shape)
print(y_valid.shape)


NameError: name 'x_valid' is not defined

In [8]:
x_test=x_test.reshape(x_test.shape[0], 1, x_test.shape[1])
print(x_test.shape)
print(y_test.shape)

(780, 1, 5122)
(780,)


In [ ]:
input_steps=1
hidden_dim=10
input_dim=5122
init=tf.keras.initializers.he_uniform()
reg=tf.keras.regularizers.l2(l=0.005)
droprate = 0.5
act='relu'

L_model = tf.keras.Sequential(
[
tf.keras.layers.Input(shape=(input_steps, input_dim)),
tf.keras.layers.GRU(hidden_dim ,return_state=False, kernel_regularizer=reg),
tf.keras.layers.Dropout(droprate),
#tf.keras.layers.Dense(300, activation=act),
tf.keras.layers.Dense(30),
tf.keras.layers.LeakyReLU(alpha=0.05),
tf.keras.layers.Dropout(droprate),
tf.keras.layers.Dense(10),
tf.keras.layers.LeakyReLU(alpha=0.05),
tf.keras.layers.Dropout(droprate),
tf.keras.layers.GaussianNoise(0.05),
tf.keras.layers.Dense(198, activation='softmax')])


L_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
            #optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001),
              loss= 'sparse_categorical_crossentropy',
              metrics= ['accuracy'])

In [ ]:
gc.collect()

In [10]:

##러닝
current_cost_train = 1000 #초기 값으로 높은 값 부여
current_cost_valid = 1000 #초기 값으로 높은 값 부여
known_best = 1000 #초기 값으로 높은 값 부여
full_step = 11
best_model=0
acc=0
trainValues=[]
validValues=[]

#TEST 828, 1548
#TRAIN 0, 828
for file in range(829): #개의 파일
    if file!=None and file!=30:
        x_train, y_train=batch_generater(file, scaler)
        x_train=x_train[10:]
        y_train=y_train[10:]
        x_train=x_train.reshape(x_train.shape[0], 1, x_train.shape[1])
        #print(x_train.shape)
        #print(y_train.shape)
        step=8
        for i in range(step):#파일 한 개당 N회 학습
            train_hist = L_model.fit(x_train, y_train, batch_size=100, verbose=1)
            loss, acc = variable_extracter(train_hist)

            gc.collect()

            if (i==step-1): #매 n번째 학습 성능 값 출력
                print("[file: {}]'s final cost = {}   acc={}".format(file, loss, acc)) 
                if (file >= 0): #1000번째부터 검증데이터로 모델 성능 평가
                    current_cost_valid = L_model.evaluate(x_valid, y_valid,batch_size=300, verbose=0)
                    gc.collect()
                    trainValues.append(loss) #학습 곡선을 출력하기 위해 현재 값 저장
                    validValues.append(current_cost_valid[0]) #검증 곡선을 출력하기위해 현재 값 저장
                    print('cost valid: {}, cost: {}'.format(current_cost_valid[0], current_cost_valid[1]))
                    if (current_cost_valid[0] < known_best): #검증 성능이 이전보다 좋을 경우 현재 모델 저장
                        known_best = current_cost_valid[0]
                        best_model=L_model
                        print("New best known cost = {}, acc={}".format(known_best, current_cost_valid[1]))

                #print("학습루틴 {}회 종료".format(file))


##
print("--- 학습 종료 ---")

last_model=L_model
label_model=best_model

Train on 590 samples
590/590 [==============================] - 2s 4ms/sample - loss: 5.6069 - accuracy: 0.0000e+00
Train on 590 samples
590/590 [==============================] - 0s 127us/sample - loss: 5.5815 - accuracy: 0.0017
Train on 590 samples
590/590 [==============================] - 0s 129us/sample - loss: 5.5372 - accuracy: 0.0017
Train on 590 samples
590/590 [==============================] - 0s 127us/sample - loss: 5.5246 - accuracy: 0.0068
Train on 590 samples
590/590 [==============================] - 0s 149us/sample - loss: 5.5010 - accuracy: 0.0068
Train on 590 samples
590/590 [==============================] - 0s 127us/sample - loss: 5.4828 - accuracy: 0.0119
Train on 590 samples
590/590 [==============================] - 0s 127us/sample - loss: 5.4366 - accuracy: 0.0153
Train on 590 samples
590/590 [==============================] - 0s 130us/sample - loss: 5.4259 - accuracy: 0.0136
[file: 0]'s final cost = 5.425857786404884   acc=0.013559321872889996
cost valid: 5.50

590/590 [==============================] - 0s 127us/sample - loss: 5.2048 - accuracy: 0.0000e+00
Train on 590 samples
590/590 [==============================] - 0s 122us/sample - loss: 5.1847 - accuracy: 0.0017
Train on 590 samples
590/590 [==============================] - 0s 124us/sample - loss: 5.1437 - accuracy: 0.0000e+00
Train on 590 samples
590/590 [==============================] - 0s 124us/sample - loss: 5.1419 - accuracy: 0.0051
Train on 590 samples
590/590 [==============================] - 0s 125us/sample - loss: 5.1180 - accuracy: 0.0136
Train on 590 samples
590/590 [==============================] - 0s 125us/sample - loss: 5.1225 - accuracy: 0.0068
Train on 590 samples
590/590 [==============================] - 0s 125us/sample - loss: 5.0804 - accuracy: 0.0288
[file: 15]'s final cost = 5.080432665550102   acc=0.02881355956196785
cost valid: 5.273317439215524, cost: 0.0
Train on 590 samples
590/590 [==============================] - 0s 122us/sample - loss: 5.1185 - accurac

590/590 [==============================] - 0s 132us/sample - loss: 5.3029 - accuracy: 0.0000e+00
Train on 590 samples
590/590 [==============================] - 0s 127us/sample - loss: 5.2662 - accuracy: 0.0000e+00
Train on 590 samples
590/590 [==============================] - 0s 127us/sample - loss: 5.2327 - accuracy: 0.0000e+00
Train on 590 samples
590/590 [==============================] - 0s 125us/sample - loss: 5.1993 - accuracy: 0.0000e+00
Train on 590 samples
590/590 [==============================] - 0s 127us/sample - loss: 5.2045 - accuracy: 0.0000e+00
Train on 590 samples
590/590 [==============================] - 0s 127us/sample - loss: 5.1660 - accuracy: 0.0000e+00
Train on 590 samples
590/590 [==============================] - 0s 127us/sample - loss: 5.1504 - accuracy: 0.0000e+00
Train on 590 samples
590/590 [==============================] - 0s 127us/sample - loss: 5.0947 - accuracy: 0.0000e+00
[file: 23]'s final cost = 5.0946843664524915   acc=0.0
cost valid: 5.24815991

590/590 [==============================] - 0s 127us/sample - loss: 5.0963 - accuracy: 0.0068
Train on 590 samples
590/590 [==============================] - 0s 149us/sample - loss: 5.0875 - accuracy: 0.0119
Train on 590 samples
590/590 [==============================] - 0s 127us/sample - loss: 5.0769 - accuracy: 0.0186
Train on 590 samples
590/590 [==============================] - 0s 129us/sample - loss: 5.0676 - accuracy: 0.0220
Train on 590 samples
590/590 [==============================] - 0s 132us/sample - loss: 5.0653 - accuracy: 0.0322
Train on 590 samples
590/590 [==============================] - 0s 127us/sample - loss: 5.0542 - accuracy: 0.0475
[file: 39]'s final cost = 5.054214938212249   acc=0.047457627952098846
cost valid: 5.226644652230399, cost: 0.0476190485060215
New best known cost = 5.226644652230399, acc=0.0476190485060215
Train on 590 samples
590/590 [==============================] - 0s 134us/sample - loss: 5.1628 - accuracy: 0.0000e+00
Train on 590 samples
590/590

cost valid: 5.23215743473598, cost: 0.0
Train on 590 samples
590/590 [==============================] - 0s 132us/sample - loss: 4.5389 - accuracy: 0.1322
Train on 590 samples
590/590 [==============================] - 0s 129us/sample - loss: 4.5374 - accuracy: 0.1119
Train on 590 samples
590/590 [==============================] - 0s 129us/sample - loss: 4.4934 - accuracy: 0.1831
Train on 590 samples
590/590 [==============================] - 0s 127us/sample - loss: 4.4852 - accuracy: 0.2153
Train on 590 samples
590/590 [==============================] - 0s 129us/sample - loss: 4.4173 - accuracy: 0.2390
Train on 590 samples
590/590 [==============================] - 0s 137us/sample - loss: 4.4105 - accuracy: 0.2797
Train on 590 samples
590/590 [==============================] - 0s 129us/sample - loss: 4.3795 - accuracy: 0.3102
Train on 590 samples
590/590 [==============================] - 0s 132us/sample - loss: 4.3445 - accuracy: 0.3475
[file: 47]'s final cost = 4.344505390878451   ac

...

cost valid: 4.701010874339512, cost: 0.0071428571827709675
Train on 590 samples
590/590 [==============================] - 1s 2ms/sample - loss: 4.7808 - accuracy: 0.0000e+00
Train on 590 samples
590/590 [==============================] - 1s 2ms/sample - loss: 4.7920 - accuracy: 0.0000e+00
Train on 590 samples
590/590 [==============================] - 1s 2ms/sample - loss: 4.7537 - accuracy: 0.0000e+00
Train on 590 samples
590/590 [==============================] - 1s 2ms/sample - loss: 4.7422 - accuracy: 0.0000e+00
Train on 590 samples
590/590 [==============================] - 1s 2ms/sample - loss: 4.7083 - accuracy: 0.0000e+00
Train on 590 samples
590/590 [==============================] - 1s 2ms/sample - loss: 4.7052 - accuracy: 0.0000e+00
Train on 590 samples
590/590 [==============================] - 1s 2ms/sample - loss: 4.6899 - accuracy: 0.0000e+00
Train on 590 samples
590/590 [==============================] - 1s 2ms/sample - loss: 4.6644 - accuracy: 0.0000e+00
[file: 769]'s

590/590 [==============================] - 1s 2ms/sample - loss: 5.7082 - accuracy: 0.0000e+00
Train on 590 samples
590/590 [==============================] - 1s 2ms/sample - loss: 5.6915 - accuracy: 0.0000e+00
[file: 784]'s final cost = 5.691465402053574   acc=0.0
cost valid: 4.647032737731934, cost: 0.1190476194024086
Train on 590 samples
590/590 [==============================] - 1s 2ms/sample - loss: 5.4510 - accuracy: 0.0000e+00
Train on 590 samples
590/590 [==============================] - 1s 2ms/sample - loss: 5.4537 - accuracy: 0.0000e+00
Train on 590 samples
590/590 [==============================] - 1s 2ms/sample - loss: 5.4335 - accuracy: 0.0000e+00
Train on 590 samples
590/590 [==============================] - 1s 2ms/sample - loss: 5.4235 - accuracy: 0.0000e+00
Train on 590 samples
590/590 [==============================] - 1s 2ms/sample - loss: 5.4125 - accuracy: 0.0000e+00
Train on 590 samples
590/590 [==============================] - 1s 2ms/sample - loss: 5.4042 - acc

590/590 [==============================] - 1s 2ms/sample - loss: 4.6968 - accuracy: 0.0000e+00
Train on 590 samples
590/590 [==============================] - 1s 2ms/sample - loss: 4.6788 - accuracy: 0.0000e+00
Train on 590 samples
590/590 [==============================] - 1s 2ms/sample - loss: 4.6782 - accuracy: 0.0000e+00
Train on 590 samples
590/590 [==============================] - 1s 2ms/sample - loss: 4.6834 - accuracy: 0.0000e+00
[file: 800]'s final cost = 4.683432425482798   acc=0.0
cost valid: 4.604337045124599, cost: 0.095238097012043
Train on 590 samples
590/590 [==============================] - 1s 2ms/sample - loss: 5.4069 - accuracy: 0.0000e+00
Train on 590 samples
590/590 [==============================] - 1s 2ms/sample - loss: 5.4106 - accuracy: 0.0000e+00
Train on 590 samples
590/590 [==============================] - 1s 2ms/sample - loss: 5.3955 - accuracy: 0.0000e+00
Train on 590 samples
590/590 [==============================] - 1s 2ms/sample - loss: 5.3689 - accu

590/590 [==============================] - 1s 2ms/sample - loss: 4.3973 - accuracy: 0.0000e+00
Train on 590 samples
590/590 [==============================] - 1s 2ms/sample - loss: 4.3803 - accuracy: 0.0000e+00
Train on 590 samples
590/590 [==============================] - 1s 2ms/sample - loss: 4.3569 - accuracy: 0.0000e+00
Train on 590 samples
590/590 [==============================] - 1s 2ms/sample - loss: 4.3293 - accuracy: 0.0000e+00
Train on 590 samples
590/590 [==============================] - 1s 2ms/sample - loss: 4.3044 - accuracy: 0.0000e+00
Train on 590 samples
590/590 [==============================] - 1s 2ms/sample - loss: 4.2998 - accuracy: 0.0000e+00
[file: 816]'s final cost = 4.299766702167059   acc=0.0
cost valid: 4.572170427867344, cost: 0.095238097012043
Train on 590 samples
590/590 [==============================] - 1s 2ms/sample - loss: 3.9108 - accuracy: 0.1153
Train on 590 samples
590/590 [==============================] - 1s 2ms/sample - loss: 3.8996 - accuracy

FileNotFoundError: [Errno 2] File C:\Users\ksi03\Downloads\RP_data\train\\828.csv does not exist: 'C:\\Users\\ksi03\\Downloads\\RP_data\\train\\\\828.csv'

In [13]:
import gc
gc.collect()

49

In [14]:
validValues

[5.502429962158203,
 5.430723973682949,
 5.387738943099976,
 5.350119386400495,
 5.349848406655448,
 5.317796502794538,
 5.326754297528948,
 5.315515313829694,
 5.295569862638201,
 5.287470715386527,
 5.2845538684300015,
 5.282434497560773,
 5.276312896183559,
 5.272202730178833,
 5.279687745230539,
 5.273317439215524,
 5.2828318391527445,
 5.285567760467529,
 5.2874845777239114,
 5.291344574519566,
 5.293398107801165,
 5.289329324449811,
 5.267728975840977,
 5.248159919466291,
 5.244763680866787,
 5.251122508730207,
 5.270112310137067,
 5.298806973866054,
 5.285003900527954,
 5.284527131489345,
 5.265685830797468,
 5.263403620038714,
 5.2330363137381415,
 5.23677624974932,
 5.234923022133963,
 5.233005591801235,
 5.23058431489127,
 5.228326456887381,
 5.226644652230399,
 5.231093270438058,
 5.225448369979858,
 5.226201432091849,
 5.220066445214408,
 5.224164690290179,
 5.225081545966012,
 5.23215743473598,
 5.231068679264614,
 5.221762452806745,
 5.2251459530421664,
 5.227864912578037

In [12]:
#x_train.shape[0]
#이렇게 돌린 모델과 다른 모델 비교하기
#best model과 now model 비교

ref=r"C:\\Users\ksi03\model\mod_max870_8.h5"
best_model.save(ref)
L_model.save(r"C:\\Users\ksi03\model\mod_max870_8_2.h5")

In [ ]:
ref=r"C:\\Users\ksi03\model\mod_max870_8_2.h5"
L_model=tf.keras.models.load_model(ref)
testcost=L_model.evaluate(x_test, y_test, verbose=0)
print(testcost[0])
print(testcost[1])
y_tmp=[]
y_final=[]
_=True
for num in range(828, 1548):
    if num!=1154 and num!=1168:
        
        x_raw=pd.read_csv(r'C:\Users\ksi03\Downloads\RP_data\test\\'+str(num)+".csv", sep=',', nrows=30).values
        x_raw=x_raw[10:]
        x_real=scalepca_pipeline(x_raw, scaler)
        x_real=x_real.reshape(x_real.shape[0], 1, x_real.shape[1])
        
        y_real=L_model.predict(x_real,batch_size=1, verbose=0) 
        
        y_tmp=[]
        for i in range(y_real.shape[0]):
            y_tmp.append(np.argmax((y_real[i])))
             

        y_final.append([num, int(np.argmax(np.bincount(np.array(y_tmp))))])
        print(num)  
    else:
        y_final.append([num, int(np.argmax(np.bincount(np.array(y_tmp))))])
        print(num)  
                         
            
      


5.000868508754633
0.0076923077
828
829
830
831
832
833
834
835
836
837
838
839
840
841
842
843
844
845
846
847
848
849
850
851
852
853
854
855
856
857
858
859
860
861
862
863
864
865
866
867
868
869
870
871
872
873
874
875
876
877
878
879
880
881
882
883
884
885
886
887
888
889
890
891
892
893
894
895
896
897
898
899
900
901
902
903
904
905
906
907
908
909
910
911
912
913
914
915
916
917
918
919
920
921
922
923
924
925
926
927
928
929
930
931
932
933
934
935
936
937
938
939
940
941
942
943
944
945
946
947
948
949
950
951
952
953
954
955
956
957
958
959
960
961
962
963
964
965
966
967
968
969
970
971
972
973
974
975
976
977
978
979
980
981
982
983
984
985
986
987
988
989
990
991
992
993
994
995
996
997
998
999
1000
1001
1002
1003
1004
1005
1006
1007
1008
1009
1010
1011
1012
1013
1014
1015
1016
1017
1018
1019
1020
1021
1022
1023
1024
1025
1026
1027
1028
1029
1030
1031
1032
1033
1034
1035
1036
1037
1038
1039
1040
1041
1042
1043
1044
1045
1046
1047
1048
1049
1050
1051
1052
1053
1054
1055
1

In [ ]:
y_tmp=np.array(y_final)
print(y_tmp.shape)


In [ ]:
y=pd.DataFrame(y_tmp, columns=['num','label'])
print(y)

In [ ]:
x=y.num.values
y1=y.label.values

print(x)
print(y1)

In [ ]:
y_one_hot = pd.DataFrame(tf.one_hot(y1, 198).numpy(), index=x)
y_one_hot.to_csv('y_one_hot.csv')